In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
tqdm.pandas()
from shapely.geometry import box, Polygon, Point

## Open and process files
1. CoreLogic 2016 tax year for Santa Clara County
2. Universe of SJ residential parcels

In [6]:
fp = '/oak/stanford/groups/deho/corelogic/data/final/CA/'
df = pd.read_csv(os.path.join(fp, 'SC_county_2018_Tax.txt'), delimiter='|', dtype={'FIPS CODE': str,
                                                                                  'APN (PARCEL NUMBER UNFORMATTED)': str})
df_deed = pd.read_csv(os.path.join(fp, 'SC_county_deed.txt'), delimiter='|', dtype={'FIPS CODE': str,
                                                                                  'APN UNFORMATTED': str,
                                                                                   'SALE DATE (YYYYMMDD)': str,
                                                                                    'RECORDING DATE (YYYYMMDD)': str,
                                                                                   'MORTGAGE DATE': str})

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5,12,13,14,15,20,21,22,28,31,32,34,37,38,49,52,53,54,90,109,110,114,115,116,117,119,120,122,123,124,126,127,128,129,131,132,134,135,136,139,140,141,142,143) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (14,20,21,25,32,33,34,35,36,37,38,41,49,55,56,57,60,61,62,63,64,72,73,84,87,92,93) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df['HOMESTEAD EXEMPT'] = df['HOMESTEAD EXEMPT'].fillna('N').map({'Y': 1, 'N': 0})

In [7]:
df_deed['SALE DATE (YYYYMMDD)'] = df_deed['SALE DATE (YYYYMMDD)'].apply(lambda x: f'{x[:-2]}01' if (not pd.isna(x) and x[-2:] == '00') else x)
df_deed['SALE DATE (YYYYMMDD)'] = df_deed['SALE DATE (YYYYMMDD)'].apply(lambda x: f'{x[:4]}01{x[-2:]}' if (not pd.isna(x) and x[4:6] == '00') else x)
df_deed['SALE DATE (YYYYMMDD)'] = df_deed['SALE DATE (YYYYMMDD)'].apply(lambda x: np.nan if (not pd.isna(x) and int(x[:4]) < 1900) else x)
df_deed['SALE DATE'] = pd.to_datetime(df_deed['SALE DATE (YYYYMMDD)'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:]}' if not pd.isna(x) else np.nan))

In [8]:
df_deed['RECORDING DATE (YYYYMMDD)'] = df_deed['RECORDING DATE (YYYYMMDD)'].apply(lambda x: f'{x[:-2]}01' if (not pd.isna(x) and x[-2:] == '00') else x)
df_deed['RECORDING DATE (YYYYMMDD)'] = df_deed['RECORDING DATE (YYYYMMDD)'].apply(lambda x: f'{x[:4]}01{x[-2:]}' if (not pd.isna(x) and x[4:6] == '00') else x)
df_deed['RECORDING DATE (YYYYMMDD)'] = df_deed['RECORDING DATE (YYYYMMDD)'].apply(lambda x: np.nan if (not pd.isna(x) and int(x[:4]) < 1900) else x)
df_deed['RECORDING DATE'] = pd.to_datetime(df_deed['RECORDING DATE (YYYYMMDD)'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:]}' if not pd.isna(x) else np.nan))

In [9]:
df_deed['MORTGAGE DATE'] = df_deed['MORTGAGE DATE'].apply(lambda x: f'{x[:-2]}01' if (not pd.isna(x) and x[-2:] == '00') else x)
df_deed['MORTGAGE DATE'] = df_deed['MORTGAGE DATE'].apply(lambda x: f'{x[:4]}01{x[-2:]}' if (not pd.isna(x) and x[4:6] == '00') else x)
df_deed['MORTGAGE DATE'] = df_deed['MORTGAGE DATE'].apply(lambda x: f'{x[:4]}12{x[-2:]}' if (not pd.isna(x) and int(x[4:6]) > 12) else x)
df_deed['MORTGAGE DATE'] = df_deed['MORTGAGE DATE'].apply(lambda x: np.nan if (not pd.isna(x) and int(x[:4]) < 1900) else x)
df_deed['MORTGAGE DATE'] = pd.to_datetime(df_deed['MORTGAGE DATE'].apply(lambda x: f'{x[:4]}-{x[4:6]}-{x[6:]}' if not pd.isna(x) else np.nan))

In [10]:
df_deed = df_deed[(df_deed['RECORDING DATE'].notna()) | (df_deed['MORTGAGE DATE'].notna()) | (df_deed['SALE DATE'].notna())]

In [18]:
h = pd.to_datetime('January 1 2000')
t = pd.to_datetime(np.nan)
t1 = pd.to_datetime(np.nan)
j = pd.to_datetime('January 2, 1999')

In [22]:
min(t1, j, t)

NaT

In [8]:
pd.to_datetime(np.nan) < h

False

In [11]:
# def pick_earliest(row):
#     earliest_date = None
#     cols = ['RECORDING DATE', 'MORTGAGE DATE', 'SALE DATE']
#     for col in cols:
#         if pd.isna(row[col]):
#             cols.remove(col)
            
    
#     if row['RECORDING DATE'] < row['MORTGAGE DATE']:
#         earliest_date = row[]
df_deed['OWNERSHIP CHANGE'] = df_deed.apply(lambda row: min(row['RECORDING DATE'], row['MORTGAGE DATE'], 
                                                            row['SALE DATE']), axis=1)

In [12]:
df_deed_missing = df_deed[df_deed['OWNERSHIP CHANGE'].isna()]
df_deed_notmissing = df_deed[df_deed['OWNERSHIP CHANGE'].notna()]

In [13]:
df_deed_missing['OWNERSHIP CHANGE'] = df_deed_missing['SALE DATE']

<ipython-input-13-d7afd03fa9d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deed_missing['OWNERSHIP CHANGE'] = df_deed_missing['SALE DATE']


In [14]:
df_deed = pd.concat([df_deed_notmissing, df_deed_missing], ignore_index=True)

In [18]:
df_deed['OWNERSHIP CHANGE YEAR'] = df_deed['OWNERSHIP CHANGE'].dt.year
df_deed['OWNERSHIP CHANGE MONTH'] = df_deed['OWNERSHIP CHANGE'].dt.month

In [16]:
df_deed = df_deed[df_deed['OWNERSHIP CHANGE YEAR'] < 2016]

In [17]:
df_deed = df_deed.sort_values(['OWNERSHIP CHANGE'])

In [20]:
df_deed = df_deed.drop_duplicates(['APN UNFORMATTED'], keep='last')

In [21]:
df_deed['geometry'] = df_deed.apply(lambda row: Point(row['PROPERTY LEVEL LONGITUDE'], row['PROPERTY LEVEL LATITUDE']), axis=1)
df_deed = gpd.GeoDataFrame(df_deed)
df_deed = df_deed.set_crs('epsg:4326')

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
<ipython-input-21-3cba07b5439a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deed['geometry'] = df_deed.apply(lambda row: Point(row['PROPERTY LEVEL LONGITUDE'], row['PROPERTY LEVEL LATITUDE']), axis=1)


In [76]:
df_deed[df_deed['APN UNFORMATTED'] == '38112002'][['PROPERTY LEVEL LATITUDE',
 'PROPERTY LEVEL LONGITUDE', 'SALE DATE (YYYYMMDD)', 'SALE AMOUNT', 'SALE CODE', 
                                                   'RECORDING DATE (YYYYMMDD)',
                                                    'MORTGAGE AMOUNT', 'MORTGAGE DATE']]

,PROPERTY LEVEL LATITUDE,PROPERTY LEVEL LONGITUDE,SALE DATE (YYYYMMDD),SALE AMOUNT,SALE CODE,RECORDING DATE (YYYYMMDD),MORTGAGE AMOUNT,MORTGAGE DATE
224393,37.308928,-121.981711,NaN,NaN,NaN,NaN,NaN,NaT
378177,37.308928,-121.981711,NaN,NaN,NaN,19931012.0,NaN,NaT


In [22]:
df['geometry'] = df.apply(lambda row: Point(row['PARCEL LEVEL LONGITUDE'], row['PARCEL LEVEL LATITUDE']), axis=1)
df = gpd.GeoDataFrame(df)
df = df.set_crs('epsg:4326')

/usr/local/lib/python3.8/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [104]:
OAK_FP = '/oak/stanford/groups/deho/building_compliance/'
OUTPUT_FP = os.path.join(OAK_FP, 'outputs', 'Population-Estimates', 'outputs')
INPUT_FP = os.path.join(OAK_FP, 'outputs', 'Population-Estimates', 'inputs')

# APN - GEOID - MEDIAN INCOME ----
# * Universe of SJ res parcels assigned to each CBG
sj_parcels_cbgs = gpd.read_file(os.path.join(OUTPUT_FP, 'sj-parcels-res-cbgs'), dtype={'APN', str})

In [105]:
sj_parcels_cbgs['geometry_parcel'] = sj_parcels_cbgs['geometry']

In [106]:
informal_units = pd.read_csv('/oak/stanford/groups/deho/building_compliance/outputs/Population-Estimates/outputs/DIMeans/b3_cevents_ca12_formal_informal_sqftcom.csv',
                            dtype={'APN': str})

In [107]:
informal_units['APN'] = informal_units['APN'].str.zfill(8)

In [108]:
apn_list = informal_units['APN'].unique()

In [110]:
sj_parcels_cbgs = sj_parcels_cbgs[sj_parcels_cbgs['APN'].isin(apn_list)]

## 2. Merge

In [154]:
merged = gpd.sjoin(df, sj_parcels_cbgs, predicate='within', how='right') 

Take out edge case -- APN 26157028 sjoined with 26157027 and 26157028

In [155]:
merged = merged[merged['APN (PARCEL NUMBER UNFORMATTED)'] != '26157027']

In [156]:
merged = merged.rename(columns={'geometry': 'geometry_cl', 'geometry_parcel': 'geometry'})

In [124]:
merged.columns

Index(['index_left', 'FIPS CODE', 'APN (PARCEL NUMBER UNFORMATTED)',
       'APN SEQUENCE NUMBER)', 'COMPOSITE PROPERTY LINKAGE KEY',
       'FORMATTED APN', 'ORIGINAL APN', 'PREVIOUS PARCEL NUMBER',
       'P-ID-IRIS-FRMTD', 'ACCOUNT NUMBER',
       ...
       'PARCEL LEVEL LONGITUDE', 'SPLIT FROM PARCEL ',
       'TAX ROLL CERTIFICATION DATE', 'TAX ROLL EDITION NUMBER',
       'BEGINNING TAX YEAR DATE', 'ENDING TAX YEAR DATE', 'APN', 'GEOID',
       'geometry_cl', 'geometry'],
      dtype='object', length=161)

In [157]:
merged = merged.drop(columns=['index_left'])

In [158]:
df_deed['geometry_deed'] = df_deed['geometry']

In [159]:
merged2 = gpd.sjoin(df_deed[['geometry', 'geometry_deed', 'APN UNFORMATTED',
                             'OWNERSHIP CHANGE', 'OWNERSHIP CHANGE YEAR', 'OWNERSHIP CHANGE MONTH',
                             'SALE AMOUNT', 'SALE CODE', 'MORTGAGE AMOUNT', 
                             'SALE DATE', 'RECORDING DATE', 'MORTGAGE DATE']],
                   merged, predicate='within', how='right')

In [160]:
merged2 = merged2[merged2['APN UNFORMATTED'] != '26157027']

In [ ]:
df_deed[['geometry', 'SALE DATE', 'SALE AMOUNT', 'SALE CODE', 'MORTGAGE AMOUNT', 'MORTGAGE DATE']]

## 3. Analysis: NAs

In [161]:
cols = ['APN', 'APN (PARCEL NUMBER UNFORMATTED)', 'TOTAL VALUE CALCULATED', 
         'LAND VALUE CALCULATED', 'IMPROVEMENT VALUE CALCULATED',
        'ASSD TOTAL VALUE', 'ASSD LAND VALUE', 'ASSD IMPROVEMENT VALUE',
        'ACRES', 'LAND SQUARE FOOTAGE',
        'UNIVERSAL BUILDING SQUARE FEET',
 'BUILDING SQUARE FEET IND',
 'BUILDING  SQUARE FEET',
 'LIVING SQUARE FEET',
 'GROUND FLOOR SQUARE FEET',
 'BEDROOMS', 
        'HOMESTEAD EXEMPT', 'OWNERSHIP CHANGE', 
        'OWNERSHIP CHANGE YEAR', 'OWNERSHIP CHANGE MONTH', 'SALE AMOUNT', 'SALE CODE', 'MORTGAGE AMOUNT',
        'geometry']

In [162]:
merged_filter = merged2[cols]

In [163]:
from operator import attrgetter


In [164]:
merged_filter['YEARS SINCE LAST OWNERSHIP CHANGE'] = 2016 - merged_filter['OWNERSHIP CHANGE YEAR']

/usr/local/lib/python3.8/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [165]:
merged_filter['MONTHS SINCE LAST OWNERSHIP CHANGE'] = pd.to_datetime('January 1, 2016').to_period('M') - \
                                                    merged_filter['OWNERSHIP CHANGE'].dt.to_period('M')

In [166]:
merged_filter['MONTHS SINCE LAST OWNERSHIP CHANGE'] = merged_filter['MONTHS SINCE LAST OWNERSHIP CHANGE'].apply(lambda x: attrgetter('n')(x) if not pd.isna(x) else np.nan)

In [167]:
merged_filter['YEARS SINCE LAST OWNERSHIP CHANGE'] = 2016 - merged_filter['OWNERSHIP CHANGE YEAR']

In [168]:
na_df = pd.DataFrame(columns=['column', 'NA'])
for i in cols:
    na_df.loc[len(na_df)] = [i, merged_filter[i].isna().sum()/len(merged_filter)]

We can't use the following columns because most of them are NAs:
- MKT TOTAL VALUE
- MKT LAND VALUE
- MKT IMPROVEMENT VALUE
- GROSS SQUARE FEET
- ADJUSTED GROSS SQUARE FEET
- BASEMENT SQUARE FEET
- POOL (though i think all NAs means they don't have pools)
- PARKING SPACES
- UNITS NUMBER

## Save

In [170]:
merged_filter.to_csv('/oak/stanford/groups/deho/building_compliance/san_jose_suppl/addl_variables/corelogic_may1223.csv', index=False)

In [2]:
merged_filter = pd.read_csv('/oak/stanford/groups/deho/building_compliance/san_jose_suppl/addl_variables/corelogic_may1223.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
merged_filter[merged_filter['APN'] == '37227C01']

,APN,APN (PARCEL NUMBER UNFORMATTED),TOTAL VALUE CALCULATED,LAND VALUE CALCULATED,IMPROVEMENT VALUE CALCULATED,ASSD TOTAL VALUE,ASSD LAND VALUE,ASSD IMPROVEMENT VALUE,ACRES,LAND SQUARE FOOTAGE,...,HOMESTEAD EXEMPT,OWNERSHIP CHANGE,OWNERSHIP CHANGE YEAR,OWNERSHIP CHANGE MONTH,SALE AMOUNT,SALE CODE,MORTGAGE AMOUNT,geometry,YEARS SINCE LAST OWNERSHIP CHANGE,MONTHS SINCE LAST OWNERSHIP CHANGE
31452,37227C01,37227001,442098.0,221049.0,221049.0,442098.0,221049.0,221049.0,0.0101,441.0,...,Y,2011-10-07,2011.0,10.0,NaN,NaN,200000.0,POLYGON ((-122.0297019861342 37.30442494264316...,5.0,51.0
31453,37227C01,37227001,442098.0,221049.0,221049.0,442098.0,221049.0,221049.0,0.0101,441.0,...,Y,2004-03-03,2004.0,3.0,NaN,NaN,148000.0,POLYGON ((-122.0297019861342 37.30442494264316...,12.0,142.0
31454,37227C01,37227001,442098.0,221049.0,221049.0,442098.0,221049.0,221049.0,0.0101,441.0,...,Y,2012-09-21,2012.0,9.0,NaN,NaN,NaN,POLYGON ((-122.0297019861342 37.30442494264316...,4.0,40.0
31455,37227C01,37227001,442098.0,221049.0,221049.0,442098.0,221049.0,221049.0,0.0101,441.0,...,Y,2015-03-09,2015.0,3.0,NaN,NaN,NaN,POLYGON ((-122.0297019861342 37.30442494264316...,1.0,10.0
31456,37227C01,37227001,442098.0,221049.0,221049.0,442098.0,221049.0,221049.0,0.0101,441.0,...,Y,1988-01-28,1988.0,1.0,NaN,NaN,NaN,POLYGON ((-122.0297019861342 37.30442494264316...,28.0,336.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32471,37227C01,37227032,670064.0,335032.0,335032.0,670064.0,335032.0,335032.0,0.0101,441.0,...,Y,1987-01-28,1987.0,1.0,NaN,F,NaN,POLYGON ((-122.0297019861342 37.30442494264316...,29.0,348.0
32472,37227C01,37227032,670064.0,335032.0,335032.0,670064.0,335032.0,335032.0,0.0101,441.0,...,Y,2012-05-23,2012.0,5.0,415000.0,F,NaN,POLYGON ((-122.0297019861342 37.30442494264316...,4.0,44.0
32473,37227C01,37227032,670064.0,335032.0,335032.0,670064.0,335032.0,335032.0,0.0101,441.0,...,Y,2015-02-03,2015.0,2.0,NaN,NaN,146500.0,POLYGON ((-122.0297019861342 37.30442494264316...,1.0,11.0
32474,37227C01,37227032,670064.0,335032.0,335032.0,670064.0,335032.0,335032.0,0.0101,441.0,...,Y,2011-01-19,2011.0,1.0,NaN,NaN,160000.0,POLYGON ((-122.0297019861342 37.30442494264316...,5.0,60.0


In [4]:
merged_filter['APN'].value_counts()

37227C01    1024
70449C01     400
46451C01     289
56770C01     256
67033001     256
            ... 
43942054       1
43942055       1
43942056       1
43942057       1
74235126       1
Name: APN, Length: 158952, dtype: int64

## Explore Deed Data

Do the APNs match? 97.8% of the APNs match via the merge

In [7]:

w3 = open(f'/oak/stanford/groups/deho/corelogic/data/final/CA/SC_county_deed.txt', 'w')

with open('/oak/stanford/groups/deho/corelogic/data/raw/deed_CA/StanfordUniversity_DEED_201906_CA.txt', encoding='ISO-8859–1') as f:
    counter = 0
    for line in tqdm(f):
        if counter < 1:
            w3.write(line)
            counter += 1
        else:
            line_buffer = line.split('|')
            if line_buffer[0] == '06085':
                w3.write(line)

w3.close()

101039484it [07:56, 212035.86it/s]
